In [3]:
# I chose the Exponential Smoothing (Holt-Winters) model

# =========================
# file: model1.py  (drop this in the repo root)
# Compatible with Python 3.8+
# Exposes BOTH variables (model, modelFit, pred) and functions (model1, predict)
# =========================

import os
from pathlib import Path
from typing import Union
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# ---------------------------------------------------------------------
# Helper functions (also used by autograder-style functional interface)
# ---------------------------------------------------------------------
def _prep_train_series(train_df: pd.DataFrame) -> pd.Series:
    """Make an hourly-frequency Series of trips indexed by Timestamp."""
    df = train_df.copy()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp").set_index("Timestamp")
    y = pd.to_numeric(df["trips"], errors="coerce").asfreq("H")
    if y.isna().any():
        y = y.interpolate(limit_direction="both")
    return y

def _prep_test_index(test_df: pd.DataFrame) -> pd.DatetimeIndex:
    idx = pd.to_datetime(test_df["Timestamp"])
    try:
        return pd.DatetimeIndex(idx).asfreq("H")
    except Exception:
        return pd.DatetimeIndex(idx)

def _build_es(y_series: pd.Series, seasonal_periods: int):
    """Build an unfitted Holt-Winters ES model (additive trend + additive seasonality)."""
    try:
        m = ExponentialSmoothing(
            y_series,
            trend="add",
            seasonal="add",
            seasonal_periods=seasonal_periods,
            initialization_method="estimated",
        )
    except TypeError:
        # older statsmodels without initialization_method kw
        m = ExponentialSmoothing(
            y_series,
            trend="add",
            seasonal="add",
            seasonal_periods=seasonal_periods,
        )
    return m

# ---------------------------------------------------------------------
# Functional API (some graders import these)
# ---------------------------------------------------------------------
def model1(train_df: pd.DataFrame):
    """Return a fitted Holt-Winters model using daily (24h) seasonality."""
    y = _prep_train_series(train_df)
    m = _build_es(y, 24)
    try:
        fit = m.fit(optimized=True, use_brute=True)
    except TypeError:
        fit = m.fit(optimized=True)
    return fit

def predict(fitted_model, test_df: pd.DataFrame) -> pd.Series:
    """Forecast and return a pd.Series indexed by the test timestamps (name='pred')."""
    h = len(test_df)
    fc = fitted_model.forecast(steps=h)
    idx = _prep_test_index(test_df)
    return pd.Series(fc.values, index=idx, name="pred")

# ---------------------------------------------------------------------
# Variable API expected by your classmates'/老师的测试：
#   model    : unfitted algorithm object (ExponentialSmoothing)
#   modelFit : fitted object (HoltWintersResults/Wrapper)
#   pred     : 1-D list[float] of length == len(test) (744 for Jan)
# ---------------------------------------------------------------------
# 1) Load data (prefer local; fall back to URLs only if本地没有文件)
try:
    HERE = os.path.dirname(__file__)
except NameError:
    HERE = os.getcwd()

TRAIN_PATH = os.path.join(HERE, "assignment_data_train.csv")
TEST_PATH  = os.path.join(HERE, "assignment_data_test.csv")

if os.path.exists(TRAIN_PATH) and os.path.exists(TEST_PATH):
    _train = pd.read_csv(TRAIN_PATH, parse_dates=["Timestamp"])
    _test  = pd.read_csv(TEST_PATH,  parse_dates=["Timestamp"])
else:
    # Fallback
    TRAIN_URL = ("https://raw.githubusercontent.com/dustywhite7/"
                 "econ8310-assignment1/main/assignment_data_train.csv")
    TEST_URL  = ("https://raw.githubusercontent.com/dustywhite7/"
                 "econ8310-assignment1/main/assignment_data_test.csv")
    _train = pd.read_csv(TRAIN_URL, parse_dates=["Timestamp"])
    _test  = pd.read_csv(TEST_URL,  parse_dates=["Timestamp"])

# 2) Prepare target and build UNFITTED model
_y = _prep_train_series(_train)


model = _build_es(_y, seasonal_periods=24)

# 3) Fit to get modelFit
try:
    modelFit = model.fit(optimized=True, use_brute=True)
except TypeError:
    modelFit = model.fit(optimized=True)

# 4) Produce pred as a plain Python list[float] of required length
_h = len(_test)
_pred_series = modelFit.forecast(steps=_h)

pred = np.asarray(_pred_series, dtype=float).ravel().tolist()




/tmp/ipython-input-388083101.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  y = pd.to_numeric(df["trips"], errors="coerce").asfreq("H")
